In [11]:
# pip install pmdarima

In [1]:
from pmdarima.arima import auto_arima

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_monthly = pd.read_csv('monthly_data.csv')

df_weekly = pd.read_excel('weekly_data_v1.xlsx')

df_validate = pd.read_csv('monthly_validate.csv')

In [4]:
df_monthly['datum'] = pd.to_datetime(df_monthly['datum'])

In [5]:
df_train = df_monthly

df_predict = df_validate.copy()

In [6]:
df_predict

,Unnamed: 0,datum,RemediMax,VitaFlux,SynaptraZol,NeuroRevive,Immunoxen,RemediSol,Vitalisol,RestoraZen
0,60,2022-01-30,179.70,222.351,99.70,1660.612,295.2,23.0,386.0,41.30
1,61,2022-02-27,133.73,142.155,110.20,1001.212,249.4,12.0,226.0,69.50
2,62,2022-03-30,154.52,113.118,83.35,941.050,301.4,19.0,257.0,169.50
3,63,2022-04-29,161.39,100.165,88.10,647.650,299.4,22.0,259.0,179.10
4,64,2022-05-30,168.04,97.258,104.10,703.562,265.8,26.0,322.0,135.40
5,65,2022-06-29,151.54,101.627,103.20,610.000,193.0,25.0,142.0,156.04
6,66,2022-07-30,181.00,103.541,92.80,649.800,250.6,20.0,115.0,105.20
7,67,2022-08-30,181.91,88.269,84.20,518.100,237.0,26.0,145.0,97.30
8,68,2022-09-29,161.07,111.437,93.50,984.480,227.8,16.0,161.0,109.10
9,69,2022-10-30,44.37,37.300,20.65,295.150,86.0,7.0,37.0,11.13


In [7]:
d_forecast = {}
n=10
df_output = pd.DataFrame()

for var in df_train.columns[2:]:
    
    print(var)
    
    model = auto_arima(df_train.set_index('datum')[var])
    
    forecast, conf_int = model.predict(n_periods=n, return_conf_int=True)
    
    df_tmp_ = pd.DataFrame()
    
    
    df_tmp_['forecast'] = forecast
    df_tmp_['date'] = df_predict['datum'].values
    df_tmp_['actual'] = df_predict[var].values
    df_tmp_['model'] = 'ARIMA'
    df_tmp_['granularity'] = 'M'
    df_tmp_['variable'] = var
    
    df_output = pd.concat([df_output,df_tmp_])
    # print(forecast)
    
    d_forecast[var] = forecast    

RemediMax


/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


VitaFlux


/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


SynaptraZol


...

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


Vitalisol


/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


RestoraZen


/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


In [12]:
df_output

,forecast,date,actual,model,granularity,variable
60,148.905737,2022-01-30,179.70,ARIMA,M,RemediMax
61,149.307686,2022-02-27,133.73,ARIMA,M,RemediMax
62,149.442802,2022-03-30,154.52,ARIMA,M,RemediMax
63,149.488221,2022-04-29,161.39,ARIMA,M,RemediMax
64,149.503489,2022-05-30,168.04,ARIMA,M,RemediMax
...,...,...,...,...,...,...
65,96.915474,2022-06-29,156.04,ARIMA,M,RestoraZen
66,98.934422,2022-07-30,105.20,ARIMA,M,RestoraZen
67,96.897098,2022-08-30,97.30,ARIMA,M,RestoraZen
68,92.153013,2022-09-29,109.10,ARIMA,M,RestoraZen


In [13]:
def mape(actual,forecast):
    return np.mean(abs(actual - forecast)/actual)

In [14]:
def smape(actual,forecast):
    return np.mean(2*abs(actual-forecast)/(abs(actual)+abs(forecast)))

In [15]:
d_error = {}

d_error['variable'] = []

d_error['mape'] = []

d_error['smape'] = []


for key in d_forecast.keys():
    
    d_error['variable'].append(key)
    
    d_error['mape'].append( mape(df_predict[key],d_forecast[key].reset_index(drop=True) ))
    
    d_error['smape'].append( smape(df_predict[key], d_forecast[key].reset_index(drop=True) ) )  

In [24]:
# pd.DataFrame(d_error)

In [20]:
pd.DataFrame(d_error).to_csv('arima_error.csv')

In [22]:
df_output.to_csv('arima_output.csv')